# Analise de dados da APP Store e Google Play para o planejamento de uma aplicação

Este projeto tem como objetivo analisar as bases de dados relacionadas às plataformas de download "APP Store" e "Google Play", respectivamente encontradas nos dispositivos moveis mais populares do mundo, Iphones e Dispositivos Android. Para a analise temos como entrada dois arquivos, um de cada plataforma, com as informações acerca de seus apps disponiveis para download, tais como nome, avaliações, genero, etc. Através desses dados busca-se entender a demanda do usuário, auxiliando no planejamento de uma aplicação baseada nas tendencias do mercado.
Para o desenvolvimento dessa aplicação foi utilizada a linguagem Python, com auxilio da IDE Jupyter Notebook. A aplicação conta com aplicações basicas e intermediárias da linguagem, tais como funções e noções de programação orientada a objetos.

### Definição da classe "Dados" e seus métodos

    Para encapsular as funções que manipulam os dados, foi criada uma classe denominada de "Dados" que conta com os métodos necessários para polir e extrair as informações que precisamimos para a analise dos dados.

In [1]:
from csv import reader
class Dados():
    
    def __init__(self, nomeArquivo):
        arquivoAberto = open(nomeArquivo,encoding='utf8')
        arquivoLido = reader(arquivoAberto)
        listaCompleta = list(arquivoLido)
        self.cabecalho = listaCompleta[0]
        self.corpo = listaCompleta[1:]
    

    def imprimirDados(self, comeco, fim):
        parte_dos_dados = self.corpo[comeco:fim]    
        for app in parte_dos_dados:
            print(app)
            print('\n')
    
    def tamanhoMatrizDeDados(self):
        print('Numero de linhas:', len(self.corpo))
        print('Numero de colunas:', len(self.corpo[0]))
    
    def identificarLinhasErradas(self):
        for app in self.corpo:
            if len(app) !=len(self.cabecalho):
                print('Linha :', (self.corpo).index(app), '\n')
    
    def deletarLinha(self, linha):
        del self.corpo[linha]
    
    def identificarAppsDuplicados(self):
        appsDuplicados = []
        appsUnicos = []

        for app in self.corpo:
            nomeApp = app[0]
            if nomeApp in appsUnicos:
                appsDuplicados.append(nomeApp)
            else:
                appsUnicos.append(nomeApp)
    
        print('Quantidade de aplicativos duplicados: ', len(appsDuplicados))
        print('\n')
        print('Examplo de apps duplicados: ', appsDuplicados[:15])
    
    def identificarAppsMaiorAvaliacao(self):
        self.maiorAvaliacao = {}
        for app in self.corpo:
            nome = app[0]
            numReviews = float(app[3])
            if nome in self.maiorAvaliacao and self.maiorAvaliacao[nome] < numReviews:
                self.maiorAvaliacao[nome] = numReviews
            elif nome not in self.maiorAvaliacao:
                self.maiorAvaliacao[nome] = numReviews
    
        print('Tamanho esperado considerando a remoção dos apps duplicados:', len(self.corpo) - 1181)
        print('Tamanho atual:', len(self.maiorAvaliacao))

    def filtrarAplicativosMaiorAvaliacao(self):
        arquivoFiltrado = []
        appsJaAdicionados = []
        for app in self.corpo:
            nome = app[0]
            numReviews = float(app[3])
            if (self.maiorAvaliacao[nome] == numReviews) and (nome not in appsJaAdicionados):
                arquivoFiltrado.append(app)
                appsJaAdicionados.append(nome)

        self.corpo = arquivoFiltrado
        self.imprimirDados(0, 5)
        self.tamanhoMatrizDeDados()
    
    def appIngles(self, nomeApp):
        naoAscii = 0
        for character in nomeApp:
            if ord(character) > 127:
                naoAscii += 1
        if naoAscii > 3:
            return False
        else:
            return True
    
    def removerAppsEstrangeiros(self, indiceNome):
        appsIngles = []
        for app in self.corpo:
            nome = app[indiceNome]
            if self.appIngles(nome) == True:
                appsIngles.append(app)
        self.corpo = appsIngles
        self.imprimirDados(0, 5)
        self.tamanhoMatrizDeDados()
        
    def filtroAppsGratuitos(self, indicePreco):
        listaFiltrada = []
        for app in self.corpo:
            preco = app[indicePreco]
            if preco == '0':
                listaFiltrada.append(app)
        
        self.corpo = listaFiltrada
        self.imprimirDados(0, 5)
        self.tamanhoMatrizDeDados()
        
    def tabelaFrequencia(self, indice):
        tabela = {}
        total = 0
        for app in self.corpo:
            atributo = app[indice]
            if atributo in tabela:
                tabela[atributo] += 1
            else:
                tabela[atributo] = 1
            total += 1

        tabelaPorcentagens = {}
        for key in tabela:
            porcentagem = (tabela[key] / total) * 100
            tabelaPorcentagens[key] = porcentagem 

        return tabelaPorcentagens
    
    def tabelaDisplay(self, indice):
        tabela = self.tabelaFrequencia(indice)
        tabelaDisplay = []
        for key in tabela:
            tuplaKeyValor = (tabela[key], key)
            tabelaDisplay.append(tuplaKeyValor)

        tabelaOrdenada = sorted(tabelaDisplay, reverse = True)
        for app in tabelaOrdenada:
            print(app[1], ':', app[0])
    
    def atributoMaisPopular(self):
        maisPopular = 0
        atributoPopular = ''
        for atributo in self.dadosPopularidade:
            popularidade = self.dadosPopularidade[atributo]
            if popularidade > maisPopular:
                maisPopular = popularidade
                atributoPopular = atributo
        
        return atributoPopular
    
    def popularidadeGenCat(self, indice, tipo):
        listaAtributos = self.tabelaFrequencia(indice)
        self.dadosPopularidade = {}
        for atributo in listaAtributos:
            total = 0
            tamanho = 0
            for app in self.corpo:
                atributoApp = app[indice]
                if atributoApp == atributo:
                    if(tipo=='Genero'):
                        numDados = float(app[6])
                        total += numDados
                        tamanho += 1
                    elif(tipo=='Categoria'):
                        numDados = app[5]
                        numDados = numDados.replace(',', '')
                        numDados = numDados.replace('+', '')
                        total += float(numDados)
                        tamanho += 1
            media = total / tamanho
            self.dadosPopularidade[atributo] = media
            print(atributo, ':', self.dadosPopularidade[atributo],'\n')
        print('---------------------------------------------------')
        print('|', tipo, ' mais popular: ',self.atributoMaisPopular(),'|')
        print('-------------------------------------------------')
    
    def imprimirDadosPorCategoria(self, categoria, sistema):
        for app in self.corpo:
            if sistema == 'IOS':
                if app[-5] == categoria:
                    print(app[2], ':', app[6])
            elif sistema == 'ANDROID':   
                if app[1] == categoria:
                    print(app[0], ':', app[5])

## Abertura e leitura dos arquivos contendo os dados

In [2]:
dadosIOS = Dados('AppleStore.csv')
dadosIOS.imprimirDados(0, 5)
dadosIOS.tamanhoMatrizDeDados()

['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']


['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']


['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3.5', '4.5', '5.0.0', '4+', 'Weather', '37', '5', '3', '1']


['4', '282614216', 'eBay: Best App to Buy, Sell, Save! Online Shopping', '128512000', 'USD', '0', '262241', '649', '4', '4.5', '5.10.0', '12+', 'Shopping', '37', '5', '9', '1']


['5', '282935706', 'Bible', '92774400', 'USD', '0', '985920', '5320', '4.5', '5', '7.5.1', '4+', 'Reference', '37', '5', '45', '1']


Numero de linhas: 7197
Numero de colunas: 17


In [3]:
dadosANDROID = Dados('googleplaystore.csv')
dadosANDROID.imprimirDados(0, 5)
dadosANDROID.tamanhoMatrizDeDados()

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


Numero de linhas: 10841
Numero de colunas: 13


## Filtragem dos dados

### Remoção de apps com incosistencias

Antes de analisarmos os dados é importante cogitar a possibilidade de linhas com dados errados, que devem ser removidas afim de não "corromper" nossa analise. Para isso utilizamos o metodo "identificarLinhasErradas()" que vai comparar o numero de colunas de cada linha com o numero de colunas do cabeçalho.

In [4]:
dadosANDROID.identificarLinhasErradas()
dadosIOS.identificarLinhasErradas()

Linha : 10472 



Ao executar o comando para ambas as listas de dados apenas o arquivo com os aplicativos android apresentou uma linha "corrompida". Para elimina-la utilizaremos o metodo deletarLinha(). 

In [5]:
dadosANDROID.tamanhoMatrizDeDados()
dadosANDROID.deletarLinha(10472)
print('Deleção realizada')
dadosANDROID.tamanhoMatrizDeDados()

Numero de linhas: 10841
Numero de colunas: 13
Deleção realizada
Numero de linhas: 10840
Numero de colunas: 13


### Removendo apps duplicados

Removidas as linhas problemáticas, outro fator com que devemos nos preocupar é com aplicativos duplicados, já que queremos fazer uma analise a partir de dados unicos. Dessa forma, inicialmente devemos identificar esses aplicativos, vamos fazer isso através do método "identificarAppsDuplicados()".

In [6]:
print('Apps ANDROID duplicados: \n')
dadosANDROID.identificarAppsDuplicados()
print('----------------------------------------')
print('Apps IOS duplicados: \n')
dadosIOS.identificarAppsDuplicados()

Apps ANDROID duplicados: 

Quantidade de aplicativos duplicados:  1181


Examplo de apps duplicados:  ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']
----------------------------------------
Apps IOS duplicados: 

Quantidade de aplicativos duplicados:  0


Examplo de apps duplicados:  []


Percebemos que apenas os aplicativos do Google Play(ANDROID) apresentam apps duplicados, sendo assim, devemos eliminar as copias dos apps duplicados afim que sejam unicos na lista. Se verificarmos os dados, embora possuam o mesmo nome, alguns aplicativos duplicados possuem informações diferentes, sendo assim é necessário pensar estrategicamente em quais linhas vao ser mantidas de forma a deixar aquela que ofereça mais precisão à nossa analise. Uma das formas de fazer essa seleção é identificando, entre os aplicativos duplicados, quais tem mais avaliação, mantendo-os na lista.

In [7]:
dadosANDROID.identificarAppsMaiorAvaliacao()


Tamanho esperado considerando a remoção dos apps duplicados: 9659
Tamanho atual: 9659


Filtrados os aplicativos de maior avaliação, os temos armazenados em um dicionario, a partir deste dicionario vamos adicionar os aplicativos a nossa lista de forma que ela fique atualizada apenas com os aplicativos unicos. Faremos essa alteração a partir do metodo "filtrarAplicativosMaiorAvaliacao()".

In [8]:
dadosANDROID.filtrarAplicativosMaiorAvaliacao()

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Numero de linhas: 9659
Numero de colunas: 13


Outro fator possivel de se observar no codigo é a presença de aplicativos em outras linguas fora o ingles. Para este projeto vamos padronizar a analise levantando apenas os que estão com o titulo em inglês. Para isso foi desenvolvido um algoritmo utilizado para saber se o nome do aplicativo possui algum caracter fora do padrao ingles. No python podemos fazer isso atraves da função "ord" que identifica a posição de um caracter na memoria e o numera. Para python, os caracteres comuns na lingua inglesa sao numerados até o 127, referenciando a tabela ascii, sendo assim, consideraremos os caracteres adiante como "estrangeiros". Embora seja uma boa logica temos que usa-la com cautela ja que alguns nomes de apps possuem caracteres de estilização, que seriam confundidos pelo algoritmo com apps estrangeiros.
Para evitar que aplicativos que possuam caracteres estranhos de estilização, limitamos esse numero de caracteres a 3.

### Removendo apps "Estrangeiros"

In [9]:
print("Apps ANDROID em ingles: \n")
dadosANDROID.removerAppsEstrangeiros(0)
print("\n ----------------------------------------------------------------- \n")
print("Apps IOS em ingles: \n")
dadosIOS.removerAppsEstrangeiros(2)

Apps ANDROID em ingles: 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Numero de linhas: 9614
Numero de colunas: 13

 ------------------

### Removendo apps pagos

Por fim, uma ultima filtragem que queremos fazer é no valor dos apps, como queremos visar a quantidade de downloads do nosso app é interessante que ele seja gratuito, dessa forma vamos filtrar apenas os apps gratuitos das plataformas. O algoritmo para esse filtro é simples, basicamente as colunas de preço serão checadas app por app e comparadas ao valor '0' que indica que o app é gratuito. Esse algoritmo sera implementado no metodo "filtroAppsGratuitos()"

In [10]:
print("Lista final de APPS ANDROID: \n")
dadosANDROID.filtroAppsGratuitos(7)
print("\n ----------------------------------------------------------------- \n")
print("Lista final de APPS IOS: \n")
dadosIOS.filtroAppsGratuitos(5)

Lista final de APPS ANDROID: 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Numero de linhas: 8864
Numero de colunas: 13

 -------------

## Análise dos dados filtrados

Feita a limpeza nos dados, é necessário analisa-los para entender que tipo de aplicativo atende mais à demanda do mercado, atraindo mais usuários, e, consequentemente, mais lucro ao desenvolvedor. 
Uma das formas é observar as colunas que indicam sobre o que o aplicativo se trata. Para os dados da APP STORE temos a coluna "prime_genre", e no GOOGLE PLAY as colunas de Genre e Category. Tais atributos vão nos ajudar a identificar que tipos de aplicativos são mais desenvolvidos nas plataformas.
Para isso, será necessário um algoritmo reponsável por identificar com que frequencia certo atributo se repete na tabela. A partir da frequencia é possivel saber a porcentagem de cada elemento da coluna, podendo assim, identificar os elementos mais frequentes nas plataformas de download.

In [11]:
print("\nAPPS android ordenados com base na frequencia\n")
dadosANDROID.tabelaDisplay(1)
print("\n ----------------------------------------------------------------- \n")
print("APPS IOS ordenados com base na frequencia\n")
dadosIOS.tabelaDisplay(-5)


APPS android ordenados com base na frequencia

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661


Enquanto a APP STORE mostrou uma predominancia clara em aplicativos voltadas para o entretenimento e jogos, no Google Play vimos que isso não se repete exatamente, apresentando um vasto catalogo de aplicativos voltados em ferramentas para bem star e produtividade. 
Mas apenas verificar que tipo de app é mais desenvolvido para as plataformas não indica diretamente o tipo de app mais procurado pelos usuários. Tendo em vista isso, uma boa alternativa é saber quais dessas categorias são mais baixadas pelos usuarios, e podemos fazer isso a partir da coluna "installs" no google play, que indica a quantidade de vezes que um app foi instalado, e "rating_count_tot" na App Store, que indica a quantidade de vezes que um app foi avaliado.

In [12]:
dadosIOS.popularidadeGenCat(-5, 'Genero')

Productivity : 21028.410714285714 

Weather : 52279.892857142855 

Shopping : 26919.690476190477 

Reference : 74942.11111111111 

Finance : 31467.944444444445 

Music : 57326.530303030304 

Utilities : 18684.456790123455 

Travel : 28243.8 

Social Networking : 71548.34905660378 

Sports : 23008.898550724636 

Health & Fitness : 23298.015384615384 

Games : 22788.6696905016 

Food & Drink : 33333.92307692308 

News : 21248.023255813954 

Book : 39758.5 

Photo & Video : 28441.54375 

Entertainment : 14029.830708661417 

Business : 7491.117647058823 

Lifestyle : 16485.764705882353 

Education : 7003.983050847458 

Navigation : 86090.33333333333 

Medical : 612.0 

Catalogs : 4004.0 

---------------------------------------------------
| Genero  mais popular:  Navigation |
-------------------------------------------------


In [13]:
dadosANDROID.popularidadeGenCat(1, 'Categoria')

ART_AND_DESIGN : 1986335.0877192982 

AUTO_AND_VEHICLES : 647317.8170731707 

BEAUTY : 513151.88679245283 

BOOKS_AND_REFERENCE : 8767811.894736841 

BUSINESS : 1712290.1474201474 

COMICS : 817657.2727272727 

COMMUNICATION : 38456119.167247385 

DATING : 854028.8303030303 

EDUCATION : 1833495.145631068 

ENTERTAINMENT : 11640705.88235294 

EVENTS : 253542.22222222222 

FINANCE : 1387692.475609756 

FOOD_AND_DRINK : 1924897.7363636363 

HEALTH_AND_FITNESS : 4188821.9853479853 

HOUSE_AND_HOME : 1331540.5616438356 

LIBRARIES_AND_DEMO : 638503.734939759 

LIFESTYLE : 1437816.2687861272 

GAME : 15588015.603248259 

FAMILY : 3695641.8198090694 

MEDICAL : 120550.61980830671 

SOCIAL : 23253652.127118643 

SHOPPING : 7036877.311557789 

PHOTOGRAPHY : 17840110.40229885 

SPORTS : 3638640.1428571427 

TRAVEL_AND_LOCAL : 13984077.710144928 

TOOLS : 10801391.298666667 

PERSONALIZATION : 5201482.6122448975 

PRODUCTIVITY : 16787331.344927534 

PARENTING : 542603.6206896552 

WEATHER : 5074

Analisando friamente vemos de cara que os aplicativos do genero Navegação e da categoria Comunicação fazem mais sucesso entre os usuários, porém isso ainda não garante que são as melhores opções na hora de escolher o tipo de aplicativos que iremos criar. Algo importante de se fazer com esses dados é ver como ele se distribui entre os aplicativos desses tipos.
Dessa forma, utilizaremos o metodo imprimirDadosPorCategoria(), que basicamente filtra os aplicativos de uma certa categoria e nos mostra sua quantidade de instalações ou avaliações.

In [14]:
print("Numero de avaliações para os apps do  tipo Navegação: \n")
dadosIOS.imprimirDadosPorCategoria('Navigation', 'IOS')
print("\n Numero de avaliações para os apps do  tipo Referencia: \n")
dadosIOS.imprimirDadosPorCategoria('Reference', 'IOS')

Numero de avaliações para os apps do  tipo Navegação: 

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Geocaching® : 12811
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5
CoPilot GPS – Car Navigation & Offline Maps : 3582
Google Maps - Navigation & Transit : 154911

 Numero de avaliações para os apps do  tipo Referencia: 

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
Merriam-Webster Dictionary : 16849
Google Translate : 26786
Night Sky : 12122
WWDC : 762
Jishokun-Japanese English Dictionary & Translator : 0
教えて!goo : 0
VPN Express : 14
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
Horror Maps for Minecraft PE - Download The Scariest 

Como podemos ver, embora os aplicativos de comunicação detenham de boa parte das avaliações gerais, uma boa parte delas se concentra em aplicativos grandes e conhecidos, de forma que se torna dificil para novos apps nessa categoria conseguirem reconhecimento do publico.
Para o genero de referencia, embora muitas avaliações se concentrem na biblia e nos diciobnararios, há um maior numero de aplicativos na plataforma e que possuem numeros consideráveis, tornando-o uma opção mais vantajosa que a anterior no planejamento de um novo app.

In [15]:
print("Numero de avaliações para os apps do  tipo Comunicação: \n")
dadosANDROID.imprimirDadosPorCategoria('COMMUNICATION', 'ANDROID')
print("\n Numero de avaliações para os apps do  tipo Livros e referencia: \n")
dadosANDROID.imprimirDadosPorCategoria('BOOKS_AND_REFERENCE', 'ANDROID')

Numero de avaliações para os apps do  tipo Comunicação: 

WhatsApp Messenger : 1,000,000,000+
Messenger for SMS : 10,000,000+
My Tele2 : 5,000,000+
imo beta free calls and text : 100,000,000+
Contacts : 50,000,000+
Call Free – Free Call : 5,000,000+
Web Browser & Explorer : 5,000,000+
Browser 4G : 10,000,000+
MegaFon Dashboard : 10,000,000+
ZenUI Dialer & Contacts : 10,000,000+
Cricket Visual Voicemail : 10,000,000+
TracFone My Account : 1,000,000+
Xperia Link™ : 10,000,000+
TouchPal Keyboard - Fun Emoji & Android Keyboard : 10,000,000+
Skype Lite - Free Video Call & Chat : 5,000,000+
My magenta : 1,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Seznam.cz : 1,000,000+
Antillean Gold Telegram (original version) : 100,000+
AT&T Visual Voicemail : 10,000,000+
GMX Mail : 10,000,000+
Omlet Chat : 10,000,000+
My Vodacom SA : 5,000,000+
Microsoft Edge : 5,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and 

Test Server SMS FA : 5+
Lite for Facebook Messenger : 1,000,000+
FC Browser - Focus Privacy Browser : 1,000+
EHiN-FH conferenceapp : 100+
Carpooling FH Hagenberg : 100+
Wi-Fi Auto-connect : 1,000,000+
Talkie - Wi-Fi Calling, Chats, File Sharing : 500,000+
WeFi - Free Fast WiFi Connect & Find Wi-Fi Map : 1,000,000+
Sat-Fi : 5,000+
Portable Wi-Fi hotspot Free : 100,000+
TownWiFi | Wi-Fi Everywhere : 500,000+
Jazz Wi-Fi : 10,000+
Sat-Fi Voice : 1,000+
Free Wi-fi HotspoT : 50,000+
FN Web Radio : 10+
FNH Payment Info : 10+
MARKET FO : 100+
FO OP St-Nazaire : 100+
FO SODEXO : 100+
FO RCBT : 100+
FO Interim : 100+
FO PSA Sept-Fons : 100+
FO AIRBUS TLSE : 1,000+
FO STELIA Méaulte : 100+
FO AIRBUS Nantes : 100+
Firefox Focus: The privacy browser : 1,000,000+
FP Connect : 100+
FreedomPop Messaging Phone/SIM : 500,000+
FP Live : 10+
HipChat - beta version : 50,000+

 Numero de avaliações para os apps do  tipo Livros e referencia: 

E-Book Read - Read Book for free : 50,000+
Download free book wit

Nesse caso acontece uma situaçao semelhante à APP STORE, embora a categoria de navegação possua mais apps instalados, os downloads se concentram em aplicativos grandes e consolidados, indicando a exitencia de uma concorrencia grande que pode não ser adequada. Já para os livros e referencias temos um resultado semelhante, onde vemos um nicho muito especifico de softwares voltados para leitura ou ferramentas de gramatica, como dicionarios.

## Conclusão
Percebe-se, através da analise dos dados, que há um grande interesse dos usuários em áreas que envolvem entretenimento e livros. Dessa forma uma opção seria